In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled)


In [2]:
%matplotlib inline
import utils_modified; reload(utils_modified)
from utils_modified import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
import numpy as np
import random
import sys

In [4]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Embedding, Activation, LSTM, merge, Flatten, Dropout, Lambda
from keras.models import Model, Sequential
from keras.engine.topology import Merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.convolutional import *
from keras.utils.data_utils import get_file

In [5]:
# https://keras.io/getting-started/sequential-model-guide/

In [10]:
from keras.layers import Input, Embedding, LSTM, Dense, merge
from keras.models import Model

# data array : 20days x 10stocks
# note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(20,10), dtype='float32', name='main_input')

# a LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(32)(main_input)

output = Dense(1, activation='relu')(lstm_out)

model = Model(input=main_input, output=output)

model.compile(optimizer='rmsprop', loss='mean_squared_error')

In [11]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 20, 10)        0                                            
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 32)            5504        main_input[0][0]                 
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             33          lstm_2[0][0]                     
Total params: 5,537
Trainable params: 5,537
Non-trainable params: 0
____________________________________________________________________________________________________


In [13]:
if False:
    X = np.random.random((50,20,10))
    Y = np.random.random((50,1))
else:
    AllXs = []
    AllYs = []
    for i in range(100):
        t = (np.random.rand()-0.5)*6 # an offset for the sinus model
        Xs = []
        for stp in range(20):
            vol = 2 + math.sin(t+stp*0.2) # a market volatility
            Xs.append( np.random.randn(1,1,10)*vol ) # one slice of stock returns
        futurevol = 2 + math.sin(t+(20-1+5)*0.2)
        #print(vol, nextvol)
        AllXs.append( np.concatenate(Xs, axis=1) )
        AllYs.append( np.array([futurevol]).reshape((1,1)) )
    X = np.concatenate(AllXs, axis=0)
    Y = np.concatenate(AllYs, axis=0)

In [14]:
model.fit(X, Y, batch_size=5, nb_epoch=20)

Epoch 1/20
100/100 [==============================] - 0s - loss: 3.5871     
Epoch 2/20
100/100 [==============================] - 0s - loss: 2.7202     
Epoch 3/20
100/100 [==============================] - 0s - loss: 1.6930     
Epoch 4/20
100/100 [==============================] - 0s - loss: 0.8720     
Epoch 5/20
100/100 [==============================] - 0s - loss: 0.6307     
Epoch 6/20
100/100 [==============================] - 0s - loss: 0.5264     
Epoch 7/20
100/100 [==============================] - 0s - loss: 0.4417     
Epoch 8/20
100/100 [==============================] - 0s - loss: 0.3676     
Epoch 9/20
100/100 [==============================] - 0s - loss: 0.3137     
Epoch 10/20
100/100 [==============================] - 0s - loss: 0.2547     
Epoch 11/20
100/100 [==============================] - 0s - loss: 0.2038     
Epoch 12/20
100/100 [==============================] - 0s - loss: 0.1618     
Epoch 13/20
100/100 [==============================] - 0s - loss: 0.1237 

In [73]:
import quandl # pip install quandl
import pandas as pd

In [61]:
def qData(tick='MSFT'):
    return quandl.get("WIKI/"+tick+".4",
                      start_date="2007-01-01",
                      end_date="2016-12-31",
                      collapse="daily")

In [62]:
TICKERS = ['MSFT','JPM','INTC','DOW','KO',
             'MCD','CAT','WMT','MMM','AXP',
             'BA','GE','XOM','PG'] # JNJ ?

In [63]:
D = {}
for tckr in TICKERS:
    print(tckr)
    D[tckr] = qData(tckr)

MSFT
JPM
INTC
DOW
KO
MCD
CAT
WMT
MMM
AXP
BA
GE
XOM
PG


In [68]:
for tck in TICKERS:
    print(D[tck].shape)
    D[tck].rename(columns={'Close': tck}, inplace = True)

(2518, 1)
(2518, 1)
(2518, 1)
(2512, 1)
(2518, 1)
(2518, 1)
(2518, 1)
(2503, 1)
(2518, 1)
(2517, 1)
(2505, 1)
(2518, 1)
(2502, 1)
(2518, 1)


In [74]:
J = D[TICKERS[0]].join(D[TICKERS[1]])
for tck in TICKERS[2:]:
    J = J.join(D[tck])

In [75]:
J.head(5)

,MSFT,JPM,INTC,DOW,KO,MCD,CAT,WMT,MMM,AXP,BA,GE,XOM,PG
Date,,,,,,,,,,,,,,
2007-01-03,29.86,48.07,20.35,40.27,48.58,43.87,61.16,47.55,78.26,60.36,89.17,37.97,74.11,64.54
2007-01-04,29.81,48.19,21.17,40.09,48.60,43.54,61.00,47.78,77.95,59.92,89.53,37.75,72.72,64.05
2007-01-05,29.64,47.79,21.10,39.71,48.26,43.54,60.22,47.39,77.42,59.13,89.15,37.56,73.24,63.50
2007-01-08,29.93,47.95,21.01,39.49,48.57,43.71,60.29,47.00,77.59,59.69,88.94,37.55,72.65,63.64
2007-01-09,29.96,47.75,21.03,39.39,48.61,43.79,60.62,47.39,77.68,59.31,88.00,37.55,72.09,63.48


In [76]:
J.shape

(2518, 14)